In [ ]:
import os
os.environ['ISX'] = '0'  # Set to zero so we don't try to load the isx module
import pickle
import warnings
warnings.filterwarnings('ignore')

import numpy as np

from dewan_calcium import AUROC, plotting
from dewan_calcium.helpers import IO, EPM
from dewan_calcium.helpers.project_folder import ProjectFolder

print("Importing required packages complete!")

In [ ]:
CELL_CLASS = 'CELL_CLASS_HERE' # vglut or vgat

AUROC_NUM_PSEUDOTRIALS = 12
ENTROPY = None
TRIM_END_S = 0  # Number of seconds to remove from the end of the data; equivalent to data = data[:-(TRIM_END_S * FPS)]
## ========== CONSTANTS ========== ##
EXPERIMENT_TIME = 10 # minutes
PSEUDOTRIAL_LEN_S = 3
ENDOSCOPE_FRAMERATE = 10

file_header = f'{CELL_CLASS}-combined'

### STEP 1C: Load Project Folder

In [ ]:
# Create Project Folder to Gather and Hold all the File Paths
project_folder = ProjectFolder('EPM', combined=True)

In [ ]:
# If this is the first time the project folder has been created, move the files to the appropriate directories and then run this cell, otherwise skip this cell
project_folder.get_data()

In [ ]:
if project_folder.raw_data_dir.combined_data_path:
    if CELL_CLASS.lower() in str(project_folder.raw_data_dir.combined_data_path).lower():
        data_file = project_folder.raw_data_dir.combined_data_path[0]
    else:
        raise FileExistsError(f'No combined data file for {CELL_CLASS} exists!')

In [ ]:
with open(data_file, 'rb') as reader:
    combined_data = pickle.load(reader)

In [ ]:
# Repack Data
auroc_data = {}
cells = []
for cell_name, cell_data in combined_data.items():
    cells.append(cell_name)
    open1 = cell_data['open1']
    open2 = cell_data['open2']
    closed1 = cell_data['closed1']
    closed2 = cell_data['closed2']
    cell_pairs = {
        'open': np.hstack([open1, open2]),
        'closed': np.hstack([closed1, closed2]),
        'c1': closed1.values,
        'c2': closed2.values,
        # 'o1': open1.values,
        # 'o2': open2.values,
    }
    auroc_data[cell_name] = cell_pairs

In [ ]:
pseudotrial_means = EPM.new_get_pseudotrials(auroc_data, AUROC_NUM_PSEUDOTRIALS, 3, 10)

In [ ]:
subsampled_pseudotrials = {}

seed_sequence = np.random.SeedSequence(entropy=ENTROPY)
print(f'Using {seed_sequence.entropy} as current entropy value...')

for cell, cell_data in pseudotrial_means.items():
    subsampled_pseudotrials[cell] = EPM.subsample_pseudotrials(cell_data, AUROC_NUM_PSEUDOTRIALS, seed_sequence)

In [ ]:
auroc_returns = AUROC.pooled_EPM_auroc(subsampled_pseudotrials, ('open', 'closed'))
auroc_returns_closed = AUROC.pooled_EPM_auroc(subsampled_pseudotrials, ('c1', 'c2'))
# auroc_returns_open = AUROC.pooled_EPM_auroc(subsampled_pseudotrials, ('o1', 'o2'))

In [ ]:

EPM_df = EPM.parse_auroc_return(auroc_returns)
EPM_df_closed = EPM.parse_auroc_return(auroc_returns_closed)
# EPM_df_open = EPM.parse_auroc_return(auroc_returns_open)

folder = project_folder.analysis_dir.output_dir
# file_name = f'{file_header}HFvFM_data_output_subset.xlsx'
file_name = f'{file_header}HFvFM_data_output.xlsx'
file_path = folder.joinpath(file_name)
EPM_df.to_excel(file_path)

# file_name = f'{file_header}HFvFM_c1vc2_output_subset.xlsx'
file_name = f'{file_header}HFvFM_c1vc2_output.xlsx'
file_path = folder.joinpath(file_name)
EPM_df_closed.to_excel(file_path)

# file_name = f'{file_header}HFvFM_o1vo2_output_subset.xlsx'
# file_path = folder.joinpath(file_name)
# EPM_df_open.to_excel(file_path)
